In [1]:
import requests
import xmltodict
import urllib, urllib.request
import mysql.connector
import time
from IPython.display import clear_output
import uuid
from webcrawler.researchgate import IeeeCrawler, ResearchGateCrawler, SpringerCrawler, PubMedCrawler
import selenium as se

## INIT

In [2]:
headers = {"x-api-key": "M7HSjQNeTfai6l7JUiDZB8XYc85BHnHt3R0NXSEd"}
# Establish database connection
cnx = mysql.connector.connect(user='david', password='daviddung1993',
                              host='127.0.0.1',
                              database='computervision')
cursor = cnx.cursor()

## Fetching

In [ ]:
# Semantic Scholar API
def semantic_paper_query(text):
    link = f"https://api.semanticscholar.org/graph/v1/paper/search?query={text}"
    response = requests.get(link, headers=headers).json()
    return response

def semantic_paper_details(paper_id):
    paper_link = f"https://api.semanticscholar.org/graph/v1/paper/{paper_id}?fields=url,year,referenceCount,citationCount,publicationTypes,journal,authors,title,references.url,references.title,references.year,references.referenceCount,references.citationCount,references.authors,references.journal,references.publicationTypes,authors.name,authors.affiliations,authors.paperCount,authors.hIndex,authors.aliases"
    response = requests.get(paper_link, headers=headers).json()
    return response

def semantic_author_details(author_id):
    author_link = f"https://api.semanticscholar.org/graph/v1/author/{author_id}?fields=affiliations,name,paperCount,hIndex,aliases"
    response = requests.get(author_link, headers=headers).json()
    return response

In [ ]:
x = f"https://api.semanticscholar.org/graph/v1/paper/530a059cb48477ad1e3d4f8f4b153274c8997332/references"
response = semantic_paper_details("05c2e1ee203be217f100d2da05bdcc52004f00b6")

semantic_author_details("96665094")
response

# Fetch all the papers in the category cs.CV from 2000 - 2020

In [ ]:
YEAR_START = 2015
YEAR_END = 2015
STEP_SIZE = 10
CATEGORY = "cs.CV"

In [ ]:
def fetch_arxiv_entries(year_temp, current_index, STEP_SIZE):
    while True:
        try:
            arxiv_query = f'https://export.arxiv.org/api/query?search_query=cat:cs.CV+AND+submittedDate:[{year_temp}01010630+TO+{year_temp}12311645]&sortBy=submittedDate&start={current_index}&max_results={STEP_SIZE}'
            data = urllib.request.urlopen(arxiv_query)
            data_xml = data.read().decode('utf-8')
            data_dict = xmltodict.parse(data_xml)

            total_entries = int(data_dict['feed']['opensearch:totalResults']['#text'])
            print(f"{current_index}/{total_entries}")
            return data_dict['feed']['entry'], total_entries
        except KeyError:
            print("sleeping")
            time.sleep(3)

def store_authors(author_information, paperID):
    if not author_information:
        cursor.execute("INSERT INTO authoredBy VALUES (%s,%s)", [paperID, "0"])
        return
    for author in author_information:
        try:
            name = author["name"]
        except:
            print(author)
            raise KeyError
        affiliations = author["affiliations"]

        # Get full name
        if "." in name and author["aliases"]:
            for alias in author["aliases"]:
                if len(alias) > len(name):
                    name = alias
                if "." not in alias and len(alias.split(" ")[0]) > 2:
                    name = alias
                    break

        params = (author["authorId"], name, author["paperCount"], author["hIndex"])
        insert_author = "INSERT INTO Authors(AuthorID, Name, PaperCount, hIndex) VALUES (%s,%s,%s,%s)"
        insert_authoredBy = "INSERT INTO authoredBy VALUES (%s,%s)"

        try:
            cursor.execute(insert_author, params)
        except mysql.connector.IntegrityError as err:
            pass
        try:
            cursor.execute(insert_authoredBy, (paperID, author["authorId"]))
        except mysql.connector.IntegrityError as err:
            pass
        """
        if affiliations:
            for affiliation in affiliations:
                cursor.execute("SELECT * FROM Affiliations a WHERE a.Name=%s", [affiliation])
                entry = cursor.fetchone()
                if not entry:
                    cursor.execute("INSERT INTO Affiliations (Name) VALUES (%s)", [affiliation])
                    lastrowid = cursor.lastrowid
                else:
                    lastrowid = entry[0]
                try:
                    cursor.execute("INSERT INTO affiliatedTo VALUES (%s,%s)", [author["authorId"],lastrowid])
                except mysql.connector.IntegrityError as err:
                    pass

        else:
            try:
                cursor.execute("INSERT INTO affiliatedTo VALUES (%s,%s)", [author["authorId"],1])
            except mysql.connector.IntegrityError as err:
                pass
        """

def store_paper(archive_url, paper_response, isLeaf):
    isConference = False
    isJournalArticle = False
    isReview = False
    journal = ''
    if not paper_response['paperId']:
        cursor.execute("SELECT * FROM Papers p WHERE Title=%s", [paper_response["title"]])
        duplicate_paper = cursor.fetchall()
        if duplicate_paper:
            paper_response['paperId'] = duplicate_paper[0][0]
        else:
            paper_response['paperId'] = '11111' + uuid.uuid4().hex
    if paper_response["publicationTypes"]:
        if "JournalArticle" in paper_response["publicationTypes"]:
            isJournalArticle = True
        if "Conference" in paper_response["publicationTypes"]:
            isConference = True
        if "Review" in paper_response["publicationTypes"]:
            isReview = True
    if paper_response["journal"]:
        if "name" in paper_response["journal"]:
            journal = paper_response["journal"]["name"]
    params = [paper_response['paperId'], paper_response['title'], archive_url, paper_response['year'], CATEGORY, isReview, isConference, isJournalArticle,paper_response['referenceCount'], paper_response['citationCount'], journal, isLeaf, paper_response['url']]
    insert_paper = "INSERT INTO Papers VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    try:
        cursor.execute(insert_paper, params)
        return True
    except mysql.connector.IntegrityError as err:
        # If paper already exists check
        if isLeaf:
            return False

        check_if_leaf = "SELECT Leaf FROM Papers WHERE PaperID = %s"
        cursor.execute(check_if_leaf, [paper_response['paperId']])
        isLeafPaper = cursor.fetchone()[0]
        if isLeafPaper == isLeaf:
            return False

        update_leaf = "UPDATE Papers SET Leaf = %s WHERE PaperID = %s"
        cursor.execute(update_leaf, (isLeaf,paper_response['paperId']))
        return True

    except Exception as err:
        raise KeyError

def store_reference(paperId, referenced_by):
    try:
        insert_reference = "INSERT INTO referencedBy VALUES (%s,%s)"
        cursor.execute(insert_reference, (paperId, referenced_by))
    except mysql.connector.IntegrityError as err:
        pass

In [ ]:
for i in range(YEAR_END - YEAR_START + 1):
    year_temp = YEAR_START + i
    print(year_temp)
    if year_temp == 2015:
        current_index = 1560
    else:
        current_index = 0
    var = 0
    while True:
        # Get title of all entries in a year from arxiv
        entries, total_entries = fetch_arxiv_entries(year_temp, current_index, STEP_SIZE)
        current_index += STEP_SIZE

        # Fetch information for papers from semantic scholar
        for entry in entries:
            var += 1
            print(var)
            archive_link = entry["id"]
            entry_title = entry["title"].replace("-", " ").replace("\n", "")
            response = semantic_paper_query(entry_title)

            if not response["total"]:
                continue
            else:
                while True:
                    try:
                        # Store papers' information in database
                        paperId = response["data"][0]["paperId"]
                        cursor.execute("select * from Papers p where p.PaperID = %s",[paperId])
                        exist_paper = cursor.fetchall()
                        if exist_paper and not exist_paper[0][-2]:
                            break
                        paper_response = semantic_paper_details(paperId)
                        if paper_response["referenceCount"] > 70 or paper_response["referenceCount"] == 0 or len(paper_response["authors"]) > 10:
                            break
                        success = store_paper(archive_link, paper_response, False)

                        if not success:
                            break

                        store_authors(paper_response['authors'], paperId)
                        # Store references' information in database
                        for reference in paper_response["references"]:
                            store_paper(None, reference, True)
                            store_reference(reference["paperId"], paperId)
                            if reference["authors"]:
                                for author in reference["authors"]:
                                    if not author["authorId"]:
                                        continue
                                    author_information = semantic_author_details(author["authorId"])
                                    store_authors([author_information], reference["paperId"])
                            else:
                                store_authors(None, reference["paperId"])
                        break
                    except KeyError:
                        continue
            cnx.commit()

        if current_index > total_entries:
            break
    clear_output(wait=True)


# Fetch doi from researchgate

In [3]:
cursor.execute("select p.PaperID, p.Title from Papers p where p.DOI is NULL")
all_papers = cursor.fetchall()
len(all_papers)

67795

In [ ]:
se.

In [5]:
se.webdriver.ChromeOptions()

AttributeError: module 'selenium' has no attribute 'webdriver'

In [19]:
research_crawler = ResearchGateCrawler()

AttributeError: module 'selenium' has no attribute 'webdriver'

In [17]:
for idx,paper in enumerate(all_papers):
    if idx % 100 == 0:
        print(idx)
    cursor.execute("select distinct a.AuthorID, a.Name from Papers p,authoredBy b,Authors a where p.PaperID=%s and p.PaperID = b.PaperID and b.AuthoredByID = a.AuthorID",[paper[0]])
    authors = cursor.fetchall()

    if not authors:
        cursor.execute("UPDATE Papers p SET p.DOI='None' where p.PaperID=%s",[paper[0]])
        cnx.commit()
        continue
    first_name = authors[0][1].split()[-1]
    doi = research_crawler.extract_doi(paper[1],first_name)
    if doi:
        cursor.execute("UPDATE Papers p SET p.DOI=%s where p.PaperID=%s",[doi,paper[0]])
    else:
        cursor.execute("UPDATE Papers p SET p.DOI='NA' where p.PaperID=%s",[paper[0]])
    cnx.commit()

0


NameError: name 'research_crawler' is not defined

In [11]:
cursor.close()
cnx.close()